In [1]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from PIL import Image
import torch
google_token = '<ADD YOUR TOKEN HERE>'
model_id = "google/paligemma-3b-mix-224"
device = "cuda:0"
dtype = torch.bfloat16
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=dtype,
    device_map=device,
    revision="bfloat16",
    token=google_token
).eval()
processor = AutoProcessor.from_pretrained(model_id,
    token=google_token)

C:\Users\zeev\anaconda3\envs\projtry\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\zeev\anaconda3\envs\projtry\Lib\site-packages\transformers\models\paligemma\configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.47s/it]


In [7]:
import pandas as pd
import os
def run_model(image1, image2):
    prompt = "you are presented with 3 images. given the left most image (context) which one of the other two optional scenes is more plausible? answer only 'middle' or 'right'"
    model_inputs1 = processor(text=prompt, images=image1, return_tensors="pt").to(model.device)
    model_inputs2 = processor(text=prompt, images=image2, return_tensors="pt").to(model.device)
    input_len1 = model_inputs1["input_ids"].shape[-1]
    input_len2 = model_inputs2["input_ids"].shape[-1]
    with torch.inference_mode():
        generation = model.generate(**model_inputs1, max_new_tokens=100, do_sample=False)
        generation = generation[0][input_len1:]
        res1 = processor.decode(generation, skip_special_tokens=True)
        generation = model.generate(**model_inputs2, max_new_tokens=100, do_sample=False)
        generation = generation[0][input_len2:]
        res2 = processor.decode(generation, skip_special_tokens=True)
    return res1, res2
df = pd.DataFrame(columns=['folder', 'dataset', 'style', 'result', 'result_reversed'])
dataset = "combined_dataset"
for folder in os.listdir(dataset):
    for inner_dataset in os.listdir(os.path.join(dataset, folder)):
        for style in os.listdir(os.path.join(dataset, folder, inner_dataset)):
            image1 = Image.open(os.path.join(dataset, folder, inner_dataset, style, 'combined_image.png')).convert('RGB')
            image2 = Image.open(os.path.join(dataset, folder, inner_dataset, style, 'combined_image_reversed.png')).convert('RGB')
            res1, res2 = run_model(image1, image2)
            new_rows = pd.DataFrame([{'folder': folder, 'dataset': inner_dataset, 'style': style, 'result': res1, 'result_reversed': res2}])
            df = pd.concat([df, new_rows], ignore_index=True)
            df.to_csv('google_paligemma-3b-mix-224_result_all.csv', index=False)
    print('Done with', folder)


Done with 1
Done with 10
Done with 106
Done with 11
Done with 111
Done with 113
Done with 116
Done with 117
Done with 118
Done with 119
Done with 12
Done with 124
Done with 127
Done with 13
Done with 131
Done with 132
Done with 133
Done with 134
Done with 136
Done with 14
Done with 140
Done with 141
Done with 145
Done with 149
Done with 15
Done with 150
Done with 153
Done with 156
Done with 158
Done with 159
Done with 16
Done with 162
Done with 164
Done with 167
Done with 168
Done with 17
Done with 172
Done with 174
Done with 177
Done with 178
Done with 179
Done with 18
Done with 181
Done with 182
Done with 184
Done with 187
Done with 188
Done with 189
Done with 19
Done with 192
Done with 193
Done with 194
Done with 195
Done with 196
Done with 199
Done with 2
Done with 20
Done with 200
Done with 204
Done with 205
Done with 208
Done with 21
Done with 212
Done with 213
Done with 214
Done with 217
Done with 218
Done with 219
Done with 22
Done with 223
Done with 227
Done with 228
Done with

In [1]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5-int4', trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5-int4', trust_remote_code=True)
model.eval()

C:\Users\zeev\anaconda3\envs\projtry\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.92s/it]


MiniCPMV(
  (llm): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      

In [2]:
def run_model(image1, image2, image3):
    question = 'describe in a sentence each image. based on the context which one of the other two images is more plausible? answer "first" or "second"'
    msgs1 = [{'role': 'user', 'content': 
        ['this is the context image:',image1, 'this is the first plausible image:',image2, 'this is the second plausible image:',image3, question]
             }]
    msgs2 = [{'role': 'user', 'content': 
        ['this is the context image:',image1, 'this is the first plausible image:',image3, 'this is the second plausible image:',image2, question]
             }]
    
    res1 = model.chat(
        image=None,
        msgs=msgs1,
        tokenizer=tokenizer,
        sampling=True, # if sampling=False, beam_search will be used by default
        temperature=0.7,
        # system_prompt='' # pass system_prompt if needed
    )
    res2 = model.chat(
        image=None,
        msgs=msgs2,
        tokenizer=tokenizer,
        sampling=True, # if sampling=False, beam_search will be used by default
        temperature=0.7,
        # system_prompt='' # pass system_prompt if needed
    )
    return res1, res2

In [7]:
import pandas as pd
import os
df = pd.DataFrame(columns=['folder', 'dataset', 'style', 'result', 'result_reversed'])
dataset = "combined_dataset"
for folder in os.listdir(dataset):
    for inner_dataset in os.listdir(os.path.join(dataset, folder)):
        for style in os.listdir(os.path.join(dataset, folder, inner_dataset)):
            for file in os.listdir(os.path.join(dataset, folder, inner_dataset, style)):
                if 'premise' in file and '_' not in file:
                    image1 = Image.open(os.path.join(dataset, folder, inner_dataset, style, file)).convert('RGB').resize((512,512))
                if 'hypothesis1' in file and '_' not in file:
                    image2 = Image.open(os.path.join(dataset, folder, inner_dataset, style, file)).convert('RGB').resize((512,512))
                if 'hypothesis2' in file and '_' not in file:
                    image3 = Image.open(os.path.join(dataset, folder, inner_dataset, style, file)).convert('RGB').resize((512,512))
            res1, res2 = run_model(image1, image2, image3)
            new_rows = pd.DataFrame([{'folder': folder, 'dataset': inner_dataset, 'style': style, 'result': res1, 'result_reversed': res2}])
            df = pd.concat([df, new_rows], ignore_index=True)
            df.to_csv('openbmb_MiniCPM-Llama3-V-2_5-int4_result_all.csv', index=False)
            image1=None
            image2=None
            image3=None
    print('Done with', folder)

Done with 1
Done with 10
Done with 106
Done with 11
Done with 111
Done with 113
Done with 116
Done with 117
Done with 118
Done with 119
Done with 12
Done with 124
Done with 127
Done with 13
Done with 131
Done with 132
Done with 133
Done with 134
Done with 136
Done with 14
Done with 140
Done with 141
Done with 145
Done with 149
Done with 15
Done with 150
Done with 153
Done with 156
Done with 158
Done with 159
Done with 16
Done with 162
Done with 164
Done with 167
Done with 168
Done with 17
Done with 172
Done with 174
Done with 177
Done with 178
Done with 179
Done with 18
Done with 181
Done with 182
Done with 184
Done with 187
Done with 188
Done with 189
Done with 19
Done with 192
Done with 193
Done with 194
Done with 195
Done with 196
Done with 199
Done with 2
Done with 20
Done with 200
Done with 204
Done with 205
Done with 208
Done with 21
Done with 212
Done with 213
Done with 214
Done with 217
Done with 218
Done with 219
Done with 22
Done with 223
Done with 227
Done with 228
Done with

In [1]:
from transformers import BitsAndBytesConfig
import requests
import torch
from PIL import Image
from io import BytesIO

from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

DEVICE = "cuda:0"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceM4/idefics2-8b",
    torch_dtype=torch.float16,    
    quantization_config=quantization_config,
)
processor = AutoProcessor.from_pretrained("HuggingFaceM4/idefics2-8b")

C:\Users\zeev\anaconda3\envs\projtry\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 7/7 [00:28<00:00,  4.10s/it]
Chat templates should be in a 'chat_template.json' file but found key='chat_template' in the processor's config. Make sure to move your template to its own file.


In [2]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "This image is the context image:"},
            {"type": "image"},
            {"type": "text", "text": "This image is scenario 1:"},
            {"type": "image"},
            {"type": "text", "text": "This image is scenario 2:"},
            {"type": "image"},
            {"type": "text", "text": "Based on the context image, which scenario is more plausible?"},
        ]
    }
]
def run_model(image1, image2, image3):
    prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(text=prompt, images=[image1, image2,image3], return_tensors="pt")
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    
    # Generate
    generated_ids = model.generate(**inputs, max_new_tokens=100)
    generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
    return generated_texts[0].split('Assistant: ')[-1]

In [15]:
import pandas as pd
import os
df = pd.DataFrame(columns=['folder', 'dataset', 'style', 'result', 'result_reversed'])
dataset = "combined_dataset"
for folder in os.listdir(dataset):
    for inner_dataset in os.listdir(os.path.join(dataset, folder)):
        for style in os.listdir(os.path.join(dataset, folder, inner_dataset)): 
            print('Starting with', folder, inner_dataset, style)
            for file in os.listdir(os.path.join(dataset, folder, inner_dataset, style)):
                if 'premise' in file and '_' not in file:
                    image1 = Image.open(os.path.join(dataset, folder, inner_dataset, style, file)).convert('RGB').resize((512,512))
                if 'hypothesis1' in file and '_' not in file:
                    image2 = Image.open(os.path.join(dataset, folder, inner_dataset, style, file)).convert('RGB').resize((512,512))
                if 'hypothesis2' in file and '_' not in file:
                    image3 = Image.open(os.path.join(dataset, folder, inner_dataset, style, file)).convert('RGB').resize((512,512))
            res1 = run_model(image1, image2, image3)
            res2 = run_model(image1, image3, image2)
            new_rows = pd.DataFrame([{'folder': folder, 'dataset': inner_dataset, 'style': style, 'result': res1, 'result_reversed': res2}])
            df = pd.concat([df, new_rows], ignore_index=True)
            df.to_csv('HuggingFaceM4_idefics2-8b_result_356.csv', index=False)
            image1=None
            image2=None
            image3=None
    print('Done with', folder)

Starting with 353 VLM image dataset anime
Starting with 353 VLM image dataset paper cut
Starting with 353 VLM image dataset photorialistic
Starting with 353 VLM image dataset sketched
Done with 353


In [4]:
csv = pd.read_csv('HuggingFaceM4_idefics2-8b_result_all.csv')

In [7]:
# get all folder id's from csv
folders = set(csv['folder'].unique())

In [10]:
# change the filders ids to string
folders = [str(folder) for folder in folders]